In [18]:
import pandas as pd
import re
import os
project_path = os.path.abspath(os.path.relpath('../../../../', os.getcwd()))
chb_root_path=os.path.join(project_path,'Dataset/CHB-MIT')
out_path=os.path.join(project_path,'BilinearNetwork/Data/Constraint/Detection')
print(out_path)

In [19]:
def get_seizure_summary(chb_name):
    local_path=os.path.join(chb_root_path,chb_name)
    with open(os.path.join(local_path,'%s-summary.txt'%chb_name), 'r') as file:
        lines = file.readlines()
    data_dict = {'File Name': [], 'Seizure ID':[],'Start Time': [], 'End Time': []}

    seizure_id = 1
    def combine(file_name_,seizure_id_,start_time_,end_time_):
        
        data_dict['File Name'].append(file_name_)
        data_dict['Start Time'].append(start_time_)
        data_dict['End Time'].append(end_time_)
        data_dict['Seizure ID'].append(seizure_id_)
        

    for i in range(len(lines)):
        line = lines[i]

        # 如果这一行是文件名
        if "File Name" in line:
            file_name = line.split(": ")[1].strip()
            num_seizures = int(lines[i+3].split(": ")[1].strip())
            # 如果文件中有Seizures
            j=i+4
            for k in range(num_seizures):
                seizure_start = lines[j].split(": ")[1].strip().split(" ")[0].strip()
                seizure_end = lines[j+1].split(": ")[1].strip().split(" ")[0].strip()
                combine(file_name,seizure_id,seizure_start,seizure_end)
                seizure_id+=1
                j+=2
        else:
            continue
    del seizure_id
    return pd.DataFrame(data_dict)


def get_all_summary():
    chb_names = os.listdir(chb_root_path)
    df = pd.DataFrame()
    for chb_name in chb_names:
        df = pd.concat([df, get_seizure_summary(chb_name)])
    # 转换为int列
    df['Start Time']=df['Start Time'].astype(int)
    df['End Time']=df['End Time'].astype(int)
    return df
df=get_all_summary()


In [20]:
# 定义一个函数，遍历每一行，对于FileName，如果有重复的，只保留最早的那个
# def remove_duplicate(df):
#     df_=df.copy()
#     df_=df_.drop_duplicates(subset='File Name',keep='first')
#     df_.reset_index(inplace=True)
#     return df_
# df_=remove_duplicate(df)
# df_.drop(columns='index',inplace=True)

In [21]:
df.to_csv(os.path.join(out_path,'seizure_ictal_summary.csv'),index=False)